In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# --- Shared State ---
class LibraryState(TypedDict):
    question: str
    faq_answer: str
    checkout_info: str
    final_answer: str


# --- Agents (nodes) ---
def ClassifierAgent(state: LibraryState):
    print("ClassifierAgent ran")
    print (f"state", state)   
    q = state["question"].lower()

    # Normally you'd use an LLM; here we mock branching logic
    if "available" in q or "checkout" in q:
        return {"faq_answer": "", "checkout_info": "Book available: The Hobbit"}
    else:
        return {"faq_answer": "Library opens at 9 AM", "checkout_info": ""}


def FAQAgent(state: LibraryState):
    print("FAQAgent ran")
    if not state.get("faq_answer"):
        return {"faq_answer": "Default FAQ: Library rules apply"}
    return {}


def CheckoutAgent(state: LibraryState):
    print("CheckoutAgent ran")
    if not state.get("checkout_info"):
        return {"checkout_info": "Checkout info: Not requested"}
    return {}


def ResponseAgent(state: LibraryState):
    print("ResponseAgent ran")
    final = f"Q: {state['question']}\n"
    if state.get("faq_answer"):
        final += f"FAQ: {state['faq_answer']}\n"
    if state.get("checkout_info"):
        final += f"Checkout: {state['checkout_info']}"
    return {"final_answer": final}


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()


# --- Visualize ---
print(graph.get_graph().draw_ascii())

# --- Run ---
result = graph.invoke({"question": "Is The Hobbit available?"})
print("\n--- Final Answer ---")
print(result["final_answer"])

               +-----------+              
               | __start__ |              
               +-----------+              
                     *                    
                     *                    
                     *                    
            +-----------------+           
            | ClassifierAgent |           
            +-----------------+           
              ***          **             
             *               **           
           **                  **         
+---------------+           +----------+  
| CheckoutAgent |           | FAQAgent |  
+---------------+           +----------+  
              ***          **             
                 *       **               
                  **   **                 
             +---------------+            
             | ResponseAgent |            
             +---------------+            
                     *                    
                     *                    
           

In [ ]:
graph.get_graph().draw_png("images/agentic_ai_library.png")
print("Graph saved as agentic_ai_library.png")